# Code chunks for Exercise 1

Welcome to this Jupyter notebook collecting many code chunks that are useful for the MRS UE exercises. It presents typical Python tools for working with geo data, as also used in professional remote sensing. A focus is put on the first exercise, i.e. classifying forests with optical imagery.

## Some Python libraries you might need

A lot of libraries will be used for the exercise, but don't get intimidated by the amout of imports. They mostly fulfill small tasks. 

First, we import some popular, useful libraries.

In [ ]:
import cv2
import datetime
import numpy as np
import pandas as pd
import xarray as xar
import os

Then, we continue with in-house production software for all sorts of data processing. For more information have a look at https://github.com/TUW-GEO.

In [ ]:
from geospade.crs import SpatialRef
from veranda.io.geotiff import write_tiff

Several tasks cannot be solely managed with the existing general-purpose packages. Therefore, we have created a helper package dedicated for this lecture.

In [ ]:
from afe.add_ons import generate_tree
from afe.add_ons import OpticalDataCube
from afe.geometry import PolygonCollection

Next, we import the `scikit-learn` machine learning framework.

In [ ]:
from sklearn import svm, tree
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

Now, we import `matplotlib` for interactive plots and `roi` to draw polygons and export them as geojson files.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from ipywidgets import widgets, FloatSlider

Finally, we import our own python scripts 

and activate its `widget` backend.

In [ ]:
%matplotlib widget  
%matplotlib widget 

It is generally a good idea to run these widget-commands twice to avoid issues when changing the backend (https://matplotlib.org/2.0.2/faq/usage_faq.html#what-is-a-backend).

To allow for running the code chunks for any user, we can globally define our username here.

In [ ]:
USER = os.getcwd().split('/')[2] #This command should automatically get your username
USER

In [ ]:
from afe.contrast import auto_clip, clip, stretch
from afe.imagechoice import quicklook, display_extent
from afe.roi import roi

## Imagesearch

Since Jupyterhub does not display the QTCI images and for easier identification of the quality of your image, you can use the python script shown in the following cells.
In the first step, we have to establish the path, which leads us to the available tiles. Next we use the function `quicklook`, the parameters `tile` and `img`  should be empty at first, so that we can choose the tile and later the image. The function will ask you for an input, aslong asat least on of the parameters is empty.

In [ ]:
gen_path = f'/home/{USER}/shared/datasets/fe/data/'
rootpath = gen_path + f'sentinel2/L2A/EU010M'
image = quicklook(rootpath, tile='', img='', figsize=(9,7))

In [ ]:
image = quicklook(rootpath, tile='5', img='106', figsize=(9,7))

In [ ]:
ul_col, ul_row = 9000,2800  
col_size, row_size = 800, 800

In [ ]:
display_extent(image, ul_col=ul_col, ul_row=ul_row, col_size=col_size, row_size=row_size, title='Sub Area')

## Xarray

The loaded data is in a structure called `xarray` there are many ways to manipulate a data array and get information out of it. For this example we will create a simple 2D xarray with the coordinates `cities` and `time` and temperature regarding variables:

In [ ]:
times = pd.date_range("2014-09-06", periods=3)
cities = ["Munich", "Vienna", "Berlin", "London", "Rome"]

max_temps = np.random.uniform(low=25, high=38, size=(5, 3))
avg_temps = np.random.uniform(low=20, high=30, size=(5, 3))
min_temps = np.random.uniform(low=15, high=25, size=(5, 3))
humidity = np.random.uniform(low=15, high=100, size=(5, 3))

temp_ds = xar.Dataset(
    coords={
        "city": cities,
        "time": times,
    },
    data_vars={
        "min_temperature": (["city", "time"], min_temps),
        "avg_temperature": (["city", "time"], avg_temps),
        "max_temperature": (["city", "time"], max_temps),
        "humidity": (["city", "time"], humidity)
    }
)
temp_ds

We can define a new xarray dataset `temps_mean`: `dim` specifies the dimension to apply the mean along. `skipna` is used to signal that NaN (Not-a-Number) values should be ignored. Keep this in mind, you will have to use `skipna` in combination with another function.

In [ ]:
temps_mean = temp_ds.mean(dim="time", skipna=False)
temps_mean

We can also extract the values of a single variable:

In [ ]:
min_temps = temps_mean['min_temperature']
min_temps

xarray extensively uses the popular `numpy` library in its background.

In case you want to extract a `numpy` array from your `xarray` data, which is sometimes needed, you can do that by:
1. (optional) converting your `xarray.Dataset` to `xarray.DataArray` by calling `to_array()` , which removes your data variables but keeps all your data in an n-dimensional array.
2. accessing your `xarray.DataArray`'s `values` property

In [ ]:
temps_mean.to_array().values

## Image processing

### Stacking

With the xarray created by loading the Sentinel 2 data we cannot just yet create a meaningful image. As we have already seen in the loaded xarray we have nine different bands available for each time, x and y-coordinate.
- B02: Blue
- B03: Green
- B04: Red
- B05: Vegetation red edge
- B06: Vegetation red edge 
- B07: Vegetation red edge 
- B08: NIR
- B11: SWIR
- B12: SWIR 

We have to extract the bands we need from our xarray in order to create an rgb-image. To do this we have to stack the concerned bands with the function `np.stack`. In the following cells the stacking of the images is explained with a simple example, so that you understand it better. 

First we create a 4 by 4 pixelgrid with random values.

In [ ]:
x_pixel =4
y_pixel =4

#R Values
r_val = np.round(np.random.rand(x_pixel,y_pixel),2)
r_val[0,0]=0
r_val[x_pixel-1,y_pixel-1]=1

#G Values
g_val = np.round(np.random.rand(x_pixel,y_pixel),2)
g_val[0,0]=0
g_val[x_pixel-1,y_pixel-1]=1

#B Values
b_val = np.round(np.random.rand(x_pixel,y_pixel),2)
b_val[0,0]=0
b_val[x_pixel-1,y_pixel-1]=1

#Stacked arrays at different axis
rgb_ax2 = np.stack([r_val, g_val, b_val], axis=2)
rgb_ax0 = np.stack([r_val, g_val, b_val], axis=0)
rgb_ax1 = np.stack([r_val, g_val, b_val], axis=1)


print('Shape of R,G,B Values (x,y):',r_val.shape)
print('Shape of RGB stacke on axis=2: (x,y,z)=',rgb_ax2.shape)
print('Shape of RGB stacke on axis=1: (x,z,y)=',rgb_ax1.shape)
print('Shape of RGB stacke on axis=0: (z,x,y)=',rgb_ax0.shape)
print('Images stacked at axis 0 or 1 are not plottable! Except in some cases when x and y is the same.')

Now we plot the RGB values separately and stack them along different axis. As you can see the stacke array along axis = 0 is not possible.

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(8,7))
ax[0,0].imshow(r_val, cmap='Reds_r')
ax[0,1].imshow(g_val, cmap='Greens_r')
ax[0,2].imshow(b_val, cmap='Blues_r')

ax[1,0].imshow(rgb_ax2)
ax[1,1].imshow(rgb_ax1)
ax[1,2].imshow(rgb_ax0)


ax[0,0].set_title('R Values')
ax[0,1].set_title('G Values')
ax[0,2].set_title('B Values')
ax[1,0].set_title('RGB Values\nStacked axis = 2')
ax[1,1].set_title('RGB Values\nStacked axis = 1')
ax[1,2].set_title('RGB Values\nStacked axis = 0')

fig.suptitle('Random RGB Values\n Note: upper left corner is always 0 and lower right corner is 1')

plt.show()

### Histogram

It is very helpful creating a histogram for your images. A histogram shows the distribution of your pixel values in regards to their color intenensity, meaning low intensities bring dark pixels (black=0) and high intensities bright pixels (white=1).

To create a histogram you can use the `hist()` function. Be careful as it only accepts one dimensional arrays. The dimension of an array can be reduced using the `flatten()` function. As we want to show the distribution for each band seperately we have to extract the bands from the rgb-image first.

In [ ]:
def plot_rgb_histogram(rgb_im, ax, title, bins = 300):
    mbins = np.linspace(np.nanmin(rgb_im), np.nanmax(rgb_im), bins)
    ax.hist(rgb_im[:, :, 2].flatten(), color='blue', bins=mbins,  alpha=0.5)
    ax.hist(rgb_im[:, :, 1].flatten(), color='green', bins=mbins,  alpha=0.5)
    ax.hist(rgb_im[:, :, 0].flatten(), color='red', bins=mbins, alpha=0.5)

    ax.set_title(title)
    ax.set_xlabel('I')
    ax.set_ylabel('n')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,5))
plot_rgb_histogram(rgb_ax2, ax[1], 'Histogram of raw RGB channels', bins=20)
ax[0].imshow(rgb_ax2)

### Clip and increase contrast of images

To fix this problem we can increase the contrast of the image. To do this we implement the following functions, which will be needed for processing your image. For more details on how these functions work and why we need them to increase contrast and correctly encode the data, take a look at https://www.cg.tuwien.ac.at/courses/EinfVisComp/Skriptum/SS13/EVC-11%20Point%20Operations.pdf for a great overview.

The Contrast functions are explained in more detail in the other code chunks. Here the functions are only imported and not directly defined.


First we load an random image for exemplary use.

In [ ]:
gen_path = f'/home/{USER}/shared/datasets/fe/data/'
root_dirpath = gen_path + f'sentinel2/L2A'
dir_tree = generate_tree(root_dirpath)
filepaths = dir_tree.file_register
dc = OpticalDataCube(filepaths=filepaths)

In [ ]:
#Filter Parameter
Tile = 'E052N016T1'
Spectral_band = dc.SPECTRAL_BANDS[0:3]
Date = datetime.date(2018,5,6)

delta = 10
Date_tuple = (Date - datetime.timedelta(days=delta), Date + datetime.timedelta(days=delta))

In [ ]:
dc.filter_spatially_by_tilename(Tile, inplace=True)
dc.filter_by_dimension(Spectral_band, name="var_name", inplace=True)
dc.filter_by_dimension([Date_tuple], expressions=[('>=','<=')], name='time', inplace=True)
dc.inventory

In [ ]:
ul_col, ul_row = 9000,2800  
col_size, row_size = 800, 800

In [ ]:
loaded_data = dc.load_available_bands_by_pixels(ul_col, ul_row, col_size, row_size)
single_image = loaded_data.sel(time=Date)
single_image

We have now filtered the Datacube for our likeing and we are going to load the RGB Values and stack them.

In [ ]:
b02 = single_image['B02'].values
b03 = single_image['B03'].values
b04 = single_image['B04'].values
rgb_raw = np.stack([b04, b03, b02], axis=2)

Now we plot the image and its Histogram. as you can see the contrast in the image is terrible. 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(11,5))
plot_rgb_histogram(rgb_raw, ax[1], 'Histogram of raw RGB channels', bins=300)

ax[0].imshow(rgb_raw)
plt.show()

Now we clip the data, inorder to increase the contrast. Sets all values in I that are outside of [v_min, v_max] to the corresponding boundary, in this case defined by the percentile

In [ ]:
rgb_clipped = auto_clip(rgb_raw.copy(), percentile=0.02)

We can create a histogram with the clipped data:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,5))

plot_rgb_histogram(rgb_clipped, ax[1], 'Histogram of clipped RGB channels')
ax[0].imshow(rgb_clipped)

You can see that the histogram now has clear borders, and all the values below/above these borders have been set to the border values. You can play around clipping fewer/more values by setting the percentile value in the `auto_clip()` function. The `pooled` parameter sets if the bands get clipped individually (`=FALSE`) or as a group (`=TRUE`).

As the intensity of the values is still too low after clipping we have to stretch the values to spread between 0 and 1. We can do this by using the `stretch()` function.

In [ ]:
rgb_clipped_stretched = stretch(rgb_clipped, 0, 1, pooled=True)

Lets show the final results:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,5))
plot_rgb_histogram(rgb_clipped_stretched, ax[1], 'Histogram of raw RGB channels')
ax[0].imshow(rgb_clipped_stretched)

If the final image still has a too low contrast you can try increasing the `percentile` parameter in the `auto_clip` function or try using the `np.log` function on the raw image before clipping and stretching it.

You can see what happens when we play around with the `percentile` parameter in the `auto_clip` function or use the `np.log` function below:

In [ ]:
rgb_clipped_stretched_01 = stretch(auto_clip(rgb_raw.copy(), percentile=0.01),0,1)
rgb_clipped_stretched_05 = stretch(auto_clip(rgb_raw.copy(), percentile=0.05),0,1)
rgb_clipped_stretched_15 = stretch(auto_clip(rgb_raw.copy(), percentile=0.15),0,1)
rgb_clipped_stretched_log = stretch(auto_clip(np.log(rgb_raw.copy()), percentile=0.05),0,1)

fig,ax = plt.subplots(2,2,figsize=(10,10))
ax[0,0].imshow(rgb_clipped_stretched_01)
ax[0,0].set_title("percentile = 0.01")
ax[0,1].imshow(rgb_clipped_stretched_05)
ax[0,1].set_title("percentile = 0.05")
ax[1,0].imshow(rgb_clipped_stretched_15)
ax[1,0].set_title("percentile = 0.15")
ax[1,1].imshow(rgb_clipped_stretched_log)
ax[1,1].set_title("log and percentile = 0.05")

## Calculating NDVI

NDVI and many other indices rely on the normalized difference, represented by the function below

In [ ]:
def normalized_difference(a, b):
    return (a - b) / (a + b)

def ndvi(xar_data):
    return normalized_difference(xar_data['<band A>'], xar_data['<band B>'])

The `ndvi` function takes an `xarray.Dataset` where both `'<band A>'` and `'<band B>'` are data variables and returns an `xarray.DataArray` for the resulting NDVI. Note, that `'<band A>'` and `'<band B>'` are only placeholders. Think about which bands have to be used for computing the NDVI.

## Creating logical matrices

Generate random 100x100 matrix filled with values from 0 to 1.

In [ ]:
rand_mat = np.random.rand(100, 100)

Create a logical matrix with the same dimensions as `rand_mat`, where values greater than 0.7 are assigned `True`, the rest is `False`:

In [ ]:
log_mat = (rand_mat > 0.7)
log_mat

Such a logical matrix can then be applied to mask values in a fashionable manner.

In [ ]:
42 * True

In [ ]:
123 * False

In [ ]:
np.array([[1, 2], [3, 4]]) * np.array([[True, False], [False, True]])

In [ ]:
rand_mat * log_mat